In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import george
import corner
import emcee

In [ ]:
from justice import simulate as sim
from justice import summarize as summ
from justice import visualize as vis
from justice import xform
from justice import lightcurve
from justice.datasets import plasticc_data
from tensorflow.contrib.framework import nest

In [ ]:
lcs = plasticc_data.PlasticcDatasetLC.get_2dlcs_by_target("data/plasticc_training_data.db",53)



In [ ]:
singlecurve = lcs[15]

In [ ]:
filts={'u':300, 'g':400, 'r': 500, 'i': 600, 'z':700, 'Y':800}

I don't like this log for the record

In [ ]:
# λs = np.array([[filts[b] for t in singlecurve.bands[b].time] for b in "ugrizY"]).ravel()
# print(len(xs))
# λs = np.concatenate(λs)

# ϕs = np.array([[np.log(ϕ+200000.) for ϕ in singlecurve.bands[b].flux] for b in "ugrizY"]).ravel()
# ϕs = np.concatenate(ϕs)
# # σs = singlecurve.bands['z'].flux_err

# ins = np.array([[x,λ] for (x,λ) in zip(xs,λs)]) 
# ϕs.shape
# ins.shape

In [ ]:
kernel = george.kernels.Matern52Kernel([10.], ndim=2, axes=0) * george.kernels.ExpSquaredKernel([5.], ndim=2, axes=1)

In [ ]:
gp = george.GP(kernel)

Note to future selves: This is not a fit!

In [ ]:
ins = singlecurve.invars
ins.shape

In [ ]:
gp.compute(ins)

In [ ]:
testx = np.linspace(np.min(ins[:,1]), np.max(ins[:, 1]), 100)
testy = np.linspace(300., 1000., 20)

tests = np.array([[x,y] for x in testx for y in testy])

In [ ]:
mean, covar = gp.predict(singlecurve.outvars[:,0], tests)

In [ ]:
mean = mean.reshape([len(testx),len(testy)])


In [ ]:
gp.get_parameter_dict()

In [ ]:
plt.matshow(mean.T)

In [ ]:
samps = gp.sample_conditional(singlecurve.outvars[:,0], tests, size=1)

In [ ]:
plt.scatter(tests[:,0], tests[:, 1], c=plt.cm.get_cmap('viridis')(samps))

# Not quite working!!! 

In [ ]:
def lnprob(p):
    gp.set_parameter_vector(p)
    return gp.log_likelihood(ϕs)


In [ ]:

initial = gp.get_parameter_vector()
ndim, nwalkers = len(initial), 32
p0 = initial + 1e-8 * np.random.randn(nwalkers, ndim)
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)

print("Running burn-in...")
p0, _, _ = sampler.run_mcmc(p0, 500)
sampler.reset()

print("Running production...")
sampler.run_mcmc(p0, 1000);


In [ ]:
_ = corner.corner(sampler.flatchain, labels=['one','two'])